In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [5]:



words = open("names.txt", "r").read().splitlines()


print(len(words))

32033


In [6]:
chs = []
a = torch.zeros([27,27], dtype=torch.int32)
mapping = sorted(list(set("".join(words))))

chartoint = {x:i+1 for i,x in enumerate(mapping)} 
chartoint["."] = 0
inttochar = {x:i for i,x in chartoint.items()}  
print(chartoint)
print(inttochar)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
for w in words:
    chs = ["."] + list(w) + ["."]

    for ch1 , ch2 in zip(chs, chs[1:]):
        pointer1 = chartoint[ch1]
        pointer2 = chartoint[ch2]

        a[pointer1, pointer2] += 1


In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(a, cmap="Blues")

for i in range(27):
    for j in range(27):
        chstr = inttochar[i] + inttochar[j]
        plt.text(j, i, chstr, ha="center", va="bottom",color="gray")
        plt.text(j, i, a[i,j].item(), ha="center", va="top",color="gray")
plt.axis("off")

In [ ]:
g = torch.Generator().manual_seed(666996)
p = (a+1).float()

print(p.sum(1))
p /= p.sum(1, keepdim=True)



for i in range(100):
    string = []
    ix = 0
    out = []
    while True:

        row = p[ix]
        sample = torch.multinomial(row, num_samples=1,replacement=True, generator=g).item()
        out.append(inttochar[sample])
        
        
        ix = sample
        if ix == 0:
            string.append(out)
            break
    print("".join(string[0]))

In [ ]:
total_log_prob = 0.0
n = 0
for w in words:
    ch = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(ch, ch[1:]):

        ix1 = chartoint[ch1]
        ix2 = chartoint[ch2]

        log_prob = torch.log(p[ix1,ix2])
        n+=1
        total_log_prob += log_prob

nll = -total_log_prob
nll /= n
nll

In [7]:
# training set for the neural network

ys, xs = [], []
elements = 0
for w in words[:]:
    if(len(w)<3):
        continue
    word = ["."] + list(w) + ["."]
    for ch1, ch2,ch3,ch4 in zip(word, word[1:],word[2:],word[3:]):
        elements+=1
        intx1 = chartoint[ch1]
        intx2 = chartoint[ch2]
        intx3 = chartoint[ch3]
        intx4 = chartoint[ch4]
        xs.append([intx1,intx2,intx3])
        ys.append(intx4)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

xenc = F.one_hot(xs, num_classes=27).float()
xenc.shape


torch.Size([164015, 3, 27])

In [9]:
generator = torch.Generator().manual_seed(666996)

weights = torch.randn((27,27), requires_grad=True)

In [10]:

for i in range(1):
    logits = xenc @ weights
    count = logits.exp()
    print("count", count.shape)
    probs = count / count.sum(2, keepdim=True)
    print("probs", probs.shape)
    probs = probs.sum(1)
    print("probs2", probs.shape)
    loss = -probs[torch.arange(elements), ys].log().mean()

    print(loss)
    weights.grad = None
 
    loss.backward()

    weights.data += -50*weights.grad

print(weights)

count torch.Size([164015, 3, 27])
probs torch.Size([164015, 3, 27])
probs2 torch.Size([164015, 27])
tensor(2.3139, grad_fn=<NegBackward0>)
tensor([[-1.6884e+00,  1.2937e+00,  1.5912e-01,  3.5266e-01, -7.9957e-02,
          1.3072e+00, -6.9457e-01, -1.1177e-01,  9.8850e-01,  5.2143e-02,
         -9.4931e-02, -5.1328e-01,  2.1666e-01, -2.5192e+00,  3.6388e-01,
          1.3021e+00, -7.1283e-01,  4.7346e-01,  1.5441e+00, -1.0345e+00,
          4.1473e-01, -8.5902e-01, -5.2727e-01, -4.7250e-01, -1.5372e+00,
          2.5493e-01,  4.4675e-01],
        [ 3.7057e+00, -1.8810e+00,  1.0507e+00, -1.2255e-02,  1.2094e+00,
         -1.0060e+00, -8.2457e-01, -5.5831e-01, -3.6949e-01,  2.4656e+00,
          8.3287e-01,  8.9706e-02, -9.3172e-01,  1.2616e+00,  1.2160e+00,
         -2.5273e-01, -2.2852e+00, -5.9606e-01,  1.0906e+00, -2.0909e-01,
         -1.7364e+00, -3.6437e-01,  5.9333e-01, -8.8205e-01,  8.6381e-01,
         -3.6853e-01, -4.8722e-01],
        [-8.7541e-01,  8.1060e-01,  3.2292e-02,  

In [ ]:

# finally, sample from the 'neural net' model
# g = torch.Generator().manual_seed(214747)

for i in range(50):

  out = []
  ix = 0
  while True:

    # ----------
    # BEFORE:
    # p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ weights # predict log-counts
    counts = logits.exp()  # counts, equivalent to N
    # probabilities for next character
    p = counts / counts.sum(1, keepdims=True)
    # ----------

    ix = torch.multinomial(
        p, num_samples=1, replacement=True).item()
    out.append(inttochar[ix])
    if ix == 0:
      break
  print(''.join(out))
